In [1]:
import pymysql
import pandas as pd
import numpy as np

# Connect to the MySQL database
conn = pymysql.connect(host='localhost',
                        user='root',
                        password='1234',
                        database='eventmate')

In [2]:
registrations = pd.read_sql('SELECT * FROM registrations', conn)   
registrations

C:\Users\Lenevo\AppData\Local\Temp\ipykernel_30520\3171785457.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  registrations = pd.read_sql('SELECT * FROM registrations', conn)


,EventID,registrations,date_time
0,41BCDEFGHIJK,20,2024-04-20 11:30:00
1,41BCDEFGHIJK,11,2024-04-20 11:25:00
2,41BCDEFGHIJK,3,2024-04-20 11:20:00
3,31EFGHIJK,10,2024-04-20 11:30:00
4,31EFGHIJK,30,2024-04-20 11:25:00
5,31EFGHIJK,15,2024-04-20 11:20:00
6,34KLMNOPQ,12,2024-04-20 11:30:00
7,34KLMNOPQ,8,2024-04-20 11:25:00
8,34KLMNOPQ,1,2024-04-20 11:20:00


In [38]:
import time

# fetch current time
curr_time = pd.to_datetime("11:30:00")

registrations["date_time"] = pd.to_datetime(registrations["date_time"])

recent_registrations = registrations[registrations["date_time"] >= curr_time - pd.Timedelta(minutes = 15)]
recent_registrations


,EventID,registrations,date_time
0,41BCDEFGHIJK,20,2024-04-20 11:30:00
1,41BCDEFGHIJK,11,2024-04-20 11:25:00
2,41BCDEFGHIJK,3,2024-04-20 11:20:00
3,31EFGHIJK,10,2024-04-20 11:30:00
4,31EFGHIJK,30,2024-04-20 11:25:00
5,31EFGHIJK,15,2024-04-20 11:20:00
6,34KLMNOPQ,12,2024-04-20 11:30:00
7,34KLMNOPQ,8,2024-04-20 11:25:00
8,34KLMNOPQ,1,2024-04-20 11:20:00


In [51]:
recent_registrations["growth_value"] = 0;
recent_registrations

,EventID,registrations,date_time,growth_value
0,41BCDEFGHIJK,20,2024-04-20 11:30:00,0
1,41BCDEFGHIJK,11,2024-04-20 11:25:00,0
2,41BCDEFGHIJK,3,2024-04-20 11:20:00,0
3,31EFGHIJK,10,2024-04-20 11:30:00,0
4,31EFGHIJK,30,2024-04-20 11:25:00,0
5,31EFGHIJK,15,2024-04-20 11:20:00,0
6,34KLMNOPQ,12,2024-04-20 11:30:00,0
7,34KLMNOPQ,8,2024-04-20 11:25:00,0
8,34KLMNOPQ,1,2024-04-20 11:20:00,0


In [52]:
recent_registrations.columns

Index(['EventID', 'registrations', 'date_time', 'growth_value'], dtype='object')

In [67]:
growth_values = {
    5: 0.5,
    10: 0.3,
    15: 0.2
}

def calculate_growth_value(row, curr_time, growth_values):
    time_diff = curr_time - row["date_time"]
    if time_diff < pd.Timedelta(minutes=5):
        return row["registrations"] * growth_values[5]
    elif time_diff < pd.Timedelta(minutes=10):
        return row["registrations"] * growth_values[10]
    elif time_diff < pd.Timedelta(minutes=15):
        return row["registrations"] * growth_values[15]
    else:
        return row["growth_value"]

growth_value_df = recent_registrations.apply(calculate_growth_value, axis=1, args=(curr_time, growth_values))
recent_registrations["growth_value"] = growth_value_df


In [68]:
recent_registrations

,EventID,registrations,date_time,growth_value
0,41BCDEFGHIJK,20,2024-04-20 11:30:00,10.0
1,41BCDEFGHIJK,11,2024-04-20 11:25:00,3.3
2,41BCDEFGHIJK,3,2024-04-20 11:20:00,0.6
3,31EFGHIJK,10,2024-04-20 11:30:00,5.0
4,31EFGHIJK,30,2024-04-20 11:25:00,9.0
5,31EFGHIJK,15,2024-04-20 11:20:00,3.0
6,34KLMNOPQ,12,2024-04-20 11:30:00,6.0
7,34KLMNOPQ,8,2024-04-20 11:25:00,2.4
8,34KLMNOPQ,1,2024-04-20 11:20:00,0.2


In [74]:
growth_factor = recent_registrations.groupby("EventID").sum("growth_value")
growth_factor

,registrations,growth_value
EventID,,
31EFGHIJK,55,17.0
34KLMNOPQ,21,8.6
41BCDEFGHIJK,34,13.9


In [76]:
growth_factor = growth_factor.sort_values(by="growth_value", ascending=False).reset_index()
growth_factor[0:3]["EventID"]

0       31EFGHIJK
1    41BCDEFGHIJK
2       34KLMNOPQ
Name: EventID, dtype: object